In [1]:
import pandas as pd
import xarray as xr

from scipy import optimize as opt
from scipy import stats

In [5]:
path = "../results/prec_hist_hist_tretorno.nc"

df = xr.open_dataset( path ).to_dataframe()

# Quitamos la intensidad, año, probabilidad, tiempo de retorno, y agregamos
# columnas para los parámetros de la distribución.
df_2 = df.copy().drop( ["PROBABILIDAD", "AÑO"], axis = 1 
    ).reset_index( "TIEMPO_RETORNO" )
#df_2 = df_2.reset_index( "DURACION" )
df_3 = df_2.copy().drop( ["INTENSIDAD", "TIEMPO_RETORNO"], axis = 1).groupby(
    ["south_north", "west_east", "DURACION"] ).mean()
cols = ["GEV_C", "GEV_LOC", "GEV_SCALE", "KTEST_P"]
df_3[ cols ] = None
t = ( ( [5, 10, 25, 50, 100, 200, 500, 1000] + [None] * 23 )
    * df_2.index.get_level_values("west_east").unique().shape[0]
    * df_2.index.get_level_values("south_north").unique().shape[0]
    * df_2.index.get_level_values("DURACION").unique().shape[0] )
#df_2["DURACION"] = d
df_2["TIEMPO_RETORNO"] = t
df_2 = df_2.dropna()

df_3

LONGITUD    LATITUD GEV_C GEV_LOC GEV_SCALE  \
south_north west_east DURACION                                                  
0           0         1        -100.453476  18.135178  None    None      None   
                      2        -100.453476  18.135178  None    None      None   
                      3        -100.453476  18.135178  None    None      None   
                      4        -100.453476  18.135178  None    None      None   
                      5        -100.453476  18.135178  None    None      None   
...                                    ...        ...   ...     ...       ...   
111         131       72        -97.333252  20.628967  None    None      None   
                      84        -97.333252  20.628967  None    None      None   
                      96        -97.333252  20.628967  None    None      None   
                      108       -97.333252  20.628967  None    None      None   
                      120       -97.333252  20.628967  None    None      None   

                               KTEST_P  
south_north west_east DURACION          
0           0         1           None  
                      2           None  
                      3           None  
                      4           None  
                      5           None  
...                                ...  
111         131       72          None  
                      84          None  
                      96          None  
                      108         None  
                      120         None  

[325248 rows x 6 columns]

In [15]:
for i in df_3.index.get_level_values("south_north").unique():
    print(f"Calculando coordenada y #{i}...")
    for j in df_3.index.get_level_values("west_east").unique():
        for k in df_3.index.get_level_values("DURACION").unique():
            pass

Calculando coordenada y #0...
Calculando coordenada y #1...
Calculando coordenada y #2...
Calculando coordenada y #3...
Calculando coordenada y #4...
Calculando coordenada y #5...
Calculando coordenada y #6...
Calculando coordenada y #7...
Calculando coordenada y #8...
Calculando coordenada y #9...
Calculando coordenada y #10...
Calculando coordenada y #11...
Calculando coordenada y #12...
Calculando coordenada y #13...
Calculando coordenada y #14...
Calculando coordenada y #15...
Calculando coordenada y #16...
Calculando coordenada y #17...
Calculando coordenada y #18...
Calculando coordenada y #19...
Calculando coordenada y #20...
Calculando coordenada y #21...
Calculando coordenada y #22...
Calculando coordenada y #23...
Calculando coordenada y #24...
Calculando coordenada y #25...
Calculando coordenada y #26...
Calculando coordenada y #27...
Calculando coordenada y #28...
Calculando coordenada y #29...
Calculando coordenada y #30...
Calculando coordenada y #31...
Calculando coorden

In [116]:
path = "../temp/*.nc"

def pre(ds):
    ds = ( ds.expand_dims( ESTADISTICA = int( 
        #ds["XLAT"].encoding["source"][-4:-3] )
        1 )
    
        ).rename_dims( { "XTIME":"MES" } 
        ).rename_vars( { "XLAT": "LATITUD", "XLONG": "LONGITUD",
        "XTIME": "MES", "Pcp": "PRECIPITACION" }
        ).drop_vars( ["XTIME_bnds"] )
        )
    return ds

ds = xr.open_mfdataset( path,
            combine = "nested", 
            concat_dim = "ESTADISTICA",
            parallel = True, preprocess = pre
            )

ds["MES"] = range(1, 13)
ds["ESTADISTICA"] = range(1, 5)
#ds = ds.swap_dims( ["south_north", "west_east", "MES", "ESTADISTICA"] )

ds.to_dataframe().reorder_levels( ["south_north",
    "west_east", "MES", "ESTADISTICA"] ).sort_index(
    ).to_xarray().set_coords( ["LONGITUD", "LATITUD"] )

<xarray.Dataset>
Dimensions:        (south_north: 112, west_east: 132, MES: 12, ESTADISTICA: 4)
Coordinates:
  * south_north    (south_north) int64 0 1 2 3 4 5 6 ... 106 107 108 109 110 111
  * west_east      (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * MES            (MES) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * ESTADISTICA    (ESTADISTICA) int64 1 2 3 4
    LONGITUD       (south_north, west_east, MES, ESTADISTICA) float32 -100.5 ...
    LATITUD        (south_north, west_east, MES, ESTADISTICA) float32 18.14 ....
Data variables:
    PRECIPITACION  (south_north, west_east, MES, ESTADISTICA) float32 0.0 ......